In [1]:
import math
import re
import torch
from torch import nn
from torch.nn import functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.nn import GRU, Linear, ReLU
%matplotlib inline

In [2]:
import random
seed_value = 42

# 设置随机种子
torch.manual_seed(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
torch.cuda.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)

# 如果使用GPU，还可以设置相关的随机种子
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
data = pd.read_csv(r'基准数据集.csv')
sequences = data.Sequence
labels = data.Label.values

In [4]:
def extract_sequences_from_fasta(file_path):
    headers = []
    sequences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        current_header = ''
        current_sequence = ''
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if current_header and current_sequence:
                    headers.append(current_header)
                    sequences.append(current_sequence)

                current_header = line[1:]
                current_sequence = ''
            else:
                current_sequence += line

        # 处理最后一个序列
        if current_header and current_sequence:
            headers.append(current_header)
            sequences.append(current_sequence)

    data = {'Meta': headers, 'Sequence': sequences}
    df = pd.DataFrame(data)
    return df

# 读取FASTA文件并提取序列
file_path = r'C:\Users\26970\Desktop\package\DL\PINC-main\DataSets\TestSets\\cRNA\Zea mays.fasta'
datah = extract_sequences_from_fasta(file_path)
print(datah.shape[0])

file_path = r'C:\Users\26970\Desktop\package\DL\PINC-main\DataSets\TestSets\\ncRNA\Zea mays.fasta'
datah_nc = extract_sequences_from_fasta(file_path)
print(datah_nc.shape[0])
datah = pd.concat([datah,datah_nc]).reset_index(drop=True)

pattern = r'(?:^|\s)([^ ]+)'  # 匹配第一个空格前的非空格字符序列作为标识部分
datah['Sequence'] = [re.search(pattern, identifier).group(1) for identifier in datah['Sequence']]
label_data = [0] * 7406 + [1] * 7406

y = pd.DataFrame(label_data, columns=['Label'])
datah.reset_index(drop=True, inplace=True)
datah = datah.join(y)

sequences_te = datah.Sequence
labels_te = datah.Label.values

datah

7406
7406


,Meta,Sequence,Label
0,Zm00001d033646_T002,ATGGCCGCCGCGACAGAGAAGACGGCTGAGGACATCCGCCGCGAGC...,0
1,Zm00001d002782_T001,ATGGCCTCTCCTTCCCCTTCTTCCCCCGCCCGCGCCTCCGGCCGCC...,0
2,Zm00001d007357_T004,ATGTTTCTTAAGGAACTAGACTTACAAAAGAGTTGTGTGAAACATC...,0
3,Zm00001d008727_T002,ATGGCTCAGATCTTGCTCCACGGCACGCTCCACGCCACCATCTTCG...,0
4,Zm00001d003476_T001,ATGCACCAGGATGCACACGAGTTCTTAAATTTTCTTTTGAATGAAC...,0
...,...,...,...
14807,URS0001BC9443_4577 Zea mays mir-393 microRNA p...,CUCCAAAGGGAUCGCAUUGAUCUAACCUGCCGAUCGACGCCGACGU...,1
14808,URS00021F59AE_4577 Zea mays (maize) zm00001e01...,GGGGAAAGTCTTTGACTTGCTAGCCAGCCGTAGTAGTAATAATACG...,1
14809,URS00021F600D_4577 Zea mays (maize) zm00001e03...,ACTTACCTGGACGGGGTCGACGGGCTATCAAGAAGGCCCGTGGCCT...,1
14810,URS00021F6599_4577 Zea mays (maize) zm00001e04...,TCTTGCCTTGCGCGATCTAAGTCTAAGGAAATACAAAAATGCGCTT...,1


In [5]:
pat = re.compile('[ACGTacgt]')

def del_end_xing(text):  # 用于删除序列末尾的*
    text = pat.findall(text)
    text = [each.lower() for each in text]
    return text

x = sequences.apply(del_end_xing)
x

0        [g, a, a, a, t, t, c, t, t, t, g, g, a, g, c, ...
1        [a, a, g, g, a, a, a, a, a, a, a, a, a, a, a, ...
2        [g, t, c, a, g, a, t, t, t, g, a, a, a, c, t, ...
3        [a, t, g, a, a, c, t, t, g, c, c, t, t, t, c, ...
4        [g, g, c, a, t, c, c, c, g, t, c, c, t, t, a, ...
                               ...                        
15995    [a, t, g, g, g, c, t, c, a, a, t, a, g, c, a, ...
15996    [a, t, g, g, a, t, t, c, c, t, c, t, c, g, c, ...
15997    [a, t, g, g, c, a, a, c, t, t, t, t, g, c, c, ...
15998    [a, t, g, g, g, g, g, c, g, a, g, t, c, g, a, ...
15999    [a, t, g, t, c, a, a, g, c, a, c, a, g, c, t, ...
Name: Sequence, Length: 16000, dtype: object

In [6]:
x_te = sequences_te.apply(del_end_xing)
x_te

0        [a, t, g, g, c, c, g, c, c, g, c, g, a, c, a, ...
1        [a, t, g, g, c, c, t, c, t, c, c, t, t, c, c, ...
2        [a, t, g, t, t, t, c, t, t, a, a, g, g, a, a, ...
3        [a, t, g, g, c, t, c, a, g, a, t, c, t, t, g, ...
4        [a, t, g, c, a, c, c, a, g, g, a, t, g, c, a, ...
                               ...                        
14807    [c, c, c, a, a, a, g, g, g, a, c, g, c, a, g, ...
14808    [g, g, g, g, a, a, a, g, t, c, t, t, t, g, a, ...
14809    [a, c, t, t, a, c, c, t, g, g, a, c, g, g, g, ...
14810    [t, c, t, t, g, c, c, t, t, g, c, g, c, g, a, ...
14811    [a, c, c, t, g, g, a, c, g, t, a, g, c, g, a, ...
Name: Sequence, Length: 14812, dtype: object

In [7]:
word_set = set()

for lst in x:
    for word in lst:
        word_set.add(word)
        
word_list = list(word_set)
word_index = dict([(each, word_list.index(each) + 1) for each in word_list])

In [8]:
word_set_te = set()

for lst_te in x_te:
    for word_te in lst_te:
        word_set_te.add(word_te)
        
word_list_te = list(word_set_te)
word_index_te = dict([(each_te, word_list_te.index(each_te) + 1) for each_te in word_list_te])
word_index_te

{'t': 1, 'c': 2, 'a': 3, 'g': 4}

In [9]:
text = x.apply(lambda x: [word_index.get(word, 0) for word in x])

In [10]:
text_te = x_te.apply(lambda x_te: [word_index_te.get(word, 0) for word in x_te])
text_te

0        [3, 1, 4, 4, 2, 2, 4, 2, 2, 4, 2, 4, 3, 2, 3, ...
1        [3, 1, 4, 4, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, ...
2        [3, 1, 4, 1, 1, 1, 2, 1, 1, 3, 3, 4, 4, 3, 3, ...
3        [3, 1, 4, 4, 2, 1, 2, 3, 4, 3, 1, 2, 1, 1, 4, ...
4        [3, 1, 4, 2, 3, 2, 2, 3, 4, 4, 3, 1, 4, 2, 3, ...
                               ...                        
14807    [2, 2, 2, 3, 3, 3, 4, 4, 4, 3, 2, 4, 2, 3, 4, ...
14808    [4, 4, 4, 4, 3, 3, 3, 4, 1, 2, 1, 1, 1, 4, 3, ...
14809    [3, 2, 1, 1, 3, 2, 2, 1, 4, 4, 3, 2, 4, 4, 4, ...
14810    [1, 2, 1, 1, 4, 2, 2, 1, 1, 4, 2, 4, 2, 4, 3, ...
14811    [3, 2, 2, 1, 4, 4, 3, 2, 4, 1, 3, 4, 2, 4, 3, ...
Name: Sequence, Length: 14812, dtype: object

In [11]:
text_len = 1200

pad_text = [l + (text_len - len(l)) * [0] if len(l) < text_len else l[:text_len] for l in text]

pad_text = np.array(pad_text)

In [12]:
text_len = 1200

pad_text_te = [l + (text_len - len(l)) * [0] if len(l) < text_len else l[:text_len] for l in text_te]

pad_text_te = np.array(pad_text_te)
pad_text_te

array([[3, 1, 4, ..., 0, 0, 0],
       [3, 1, 4, ..., 0, 0, 0],
       [3, 1, 4, ..., 4, 1, 4],
       ...,
       [3, 2, 1, ..., 0, 0, 0],
       [1, 2, 1, ..., 0, 0, 0],
       [3, 2, 2, ..., 4, 3, 4]])

In [13]:
x_train, x_test, y_train, y_test = train_test_split(pad_text, labels, test_size=0.3)
# x_train = pad_text
# x_test = pad_text_te
# y_train = labels
# y_test = labels_te

In [14]:
class Mydataset(torch.utils.data.Dataset):
    def __init__(self, text_list, label_list):
        self.text_list = text_list
        self.label_list = label_list
    
    def __getitem__(self,index):
        text = torch.LongTensor(self.text_list[index])
        label = self.label_list[index]
        return text, label
    
    def __len__(self):
        return len(self.text_list)

train_ds = Mydataset(x_train, y_train)
test_ds = Mydataset(x_test, y_test)


batch_size = 16

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)

In [15]:
# embed_dim = 2 ** (int(np.log2(len(word_list) ** 0.25)) + 2)  # 经验值

embed_dim = 50
hidden_size = 20
print('embed_dim:', embed_dim)
print('hidden_size:', hidden_size)

class Net(nn.Module):
    def __init__(self, word_list, embed_dim, hidden_size, num_layers=2):
        super().__init__()
        
        self.em = nn.Embedding(len(word_list) + 1, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_size, num_layers, bidirectional=True)
        self.linear1 = nn.Linear(2 * hidden_size, 128)
        self.linear2 = nn.Linear(128, 2)
 
    def forward(self, inputs):
        x = self.em(inputs) # inputs is input, size (seq_len, batch, input_size)
        x = x.float()
        x, _ = self.gru(x) # x is outuput, size (seq_len, batch, hidden_size) 
        x = torch.sum(x, dim=0)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x

embed_dim: 50
hidden_size: 20


In [16]:
model = Net(word_list, embed_dim, hidden_size)
model = model.to('cuda')

loss = nn.CrossEntropyLoss()
loss = loss.to('cuda')

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [17]:
def fit(model, optimizer, train_dl, test_dl):
    
    tr_correct = 0  # 预测正确的个数
    tr_total = 0  # 总样本数
    tr_loss = 0
    tr_TP = 0
    tr_TN = 0
    tr_FP = 0
    tr_FN = 0
    
    model.train()  # 训练模式
    for x, y in train_dl:
        x = x.permute(1, 0)
        x, y = x.to('cuda'), y.to('cuda')
        y_pred = model(x)
        loss_value = loss(y_pred, y)
        #flood=(loss_value - 0.002).abs() + 0.002  # 洪泛函数：防止过拟合
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()
        
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            tr_correct += (y_pred == y).sum().item()
            tr_TP += ((y_pred == y) & (y == 1)).sum().item()
            tr_FN += ((y_pred != y) & (y == 1)).sum().item()
            tr_FP += ((y_pred != y) & (y == 0)).sum().item()
            tr_TN += ((y_pred == y) & (y == 0)).sum().item()
            tr_total += len(y)
            tr_loss += loss_value.item()  # 最后的loss还要除以batch数
            
    """1个epoch训练结束后，计算训练集的各个指标"""
    epoch_tr_loss = tr_loss / len(train_dl)
    epoch_tr_accuracy = tr_correct / tr_total
    epoch_tr_MCC = (tr_TP * tr_TN - tr_TP * tr_FN) / (math.sqrt((tr_TP + tr_FP) * (tr_TP + tr_FN) * (tr_TN + tr_FP) * (tr_TN + tr_FN)+ 0.01)) 
    epoch_tr_SE=tr_TP/(tr_TP+tr_FN+ 0.01) 
    epoch_tr_SPC = tr_TN / (tr_TN + tr_FP+ 0.01) 
    epoch_tr_PPV= tr_TP / (tr_TP + tr_FP+ 0.01) 
    epoch_tr_NPV= tr_TN / (tr_TN + tr_FN+ 0.01) 
    epoch_tr_recall = tr_TP / (tr_TP + tr_FN+ 0.01) 
    epoch_tr_precision = tr_TP / (tr_TP + tr_FP+ 0.01) 
    epoch_tr_F1 = (2 * epoch_tr_precision * epoch_tr_recall) / (epoch_tr_precision + epoch_tr_recall+ 0.01) 
    
    
    te_correct = 0  # 预测正确的个数
    te_total = 0  # 总样本数
    te_loss = 0
    te_TP = 0
    te_TN = 0
    te_FP = 0
    te_FN = 0
    
    model.eval()  # 评估模式
    with torch.no_grad():
        for x, y in test_dl:
            x = x.permute(1, 0)
            x, y = x.to('cuda'), y.to('cuda')
            y_pred = model(x)
            loss_value = loss(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            te_correct += (y_pred == y).sum().item()
            te_TP += ((y_pred == y) & (y == 1)).sum().item()
            te_FN += ((y_pred != y) & (y == 1)).sum().item()
            te_FP += ((y_pred != y) & (y == 0)).sum().item()
            te_TN += ((y_pred == y) & (y == 0)).sum().item()
            te_total += len(y)
            te_loss += loss_value.item()
        
    """1个epoch训练结束后，计算测试集的各个指标"""
    epoch_te_loss = te_loss / len(test_dl)
    epoch_te_accuracy = te_correct / te_total
    epoch_te_MCC = (te_TP * te_TN - te_TP * te_FN) / (math.sqrt((te_TP + te_FP) * (te_TP + te_FN) * (te_TN + te_FP) * (te_TN + te_FN)+ 0.01)) 
    epoch_te_SE=te_TP/(te_TP+te_FN + 0.01)
    epoch_te_SPC = te_TN / (te_TN + te_FP + 0.01) 
    epoch_te_PPV= te_TP / (te_TP + te_FP + 0.01) 
    epoch_te_NPV= te_TN / (te_TN + te_FN + 0.01)
    epoch_te_recall = te_TP / (te_TP + te_FN + 0.01) 
    epoch_te_precision = te_TP / (te_TP + te_FP + 0.01)
    epoch_te_F1 = (2 * epoch_te_precision * epoch_te_recall) / (epoch_te_precision + epoch_te_recall + 0.01)

    return epoch_tr_loss, epoch_tr_accuracy, epoch_tr_MCC, epoch_tr_SE,epoch_tr_F1, epoch_tr_SPC,epoch_tr_PPV, epoch_tr_NPV,epoch_te_loss, epoch_te_accuracy, epoch_te_MCC, epoch_te_SE, epoch_te_SPC,epoch_te_PPV,epoch_te_F1,epoch_te_NPV 

In [18]:
tr_loss = []
tr_accuracy = []
tr_MCC = []
tr_SE = []
tr_SPC = []
tr_PPV=[]
tr_NPV=[]
tr_AUC=[]
tr_F1=[]

te_loss = []
te_accuracy = []
te_MCC = []
te_SE = []
te_SPC = []
te_PPV=[]
te_NPV=[]
te_AUC=[]
te_F1=[]
from sklearn import metrics

In [19]:
epochs = 80

for epoch in range(epochs): 
    print(f'{epoch} : ',end = '')
    epoch_tr_loss, epoch_tr_accuracy, epoch_tr_MCC, epoch_tr_SE,epoch_tr_F1, epoch_tr_SPC,epoch_tr_PPV, epoch_tr_NPV,epoch_te_loss, epoch_te_accuracy, epoch_te_MCC, epoch_te_SE, epoch_te_SPC,epoch_te_PPV,epoch_te_F1,epoch_te_NPV= fit(model, optimizer, train_dl, test_dl)
    tr_loss.append(epoch_tr_loss)
    tr_accuracy.append(epoch_tr_accuracy)
    tr_MCC.append(epoch_tr_MCC)
    tr_SE.append(epoch_tr_SE)
    tr_SPC.append(epoch_tr_SPC)
    tr_PPV.append(epoch_tr_PPV)
    tr_NPV.append(epoch_tr_NPV)
    tr_F1.append(epoch_tr_F1)
    "tr_AUC.append(epoch_tr_AUC)"
    
    te_loss.append(epoch_te_loss)
    te_accuracy.append(epoch_te_accuracy)
    print(epoch_te_accuracy)
    te_MCC.append(epoch_te_MCC)
    te_SE.append(epoch_te_SE)
    te_SPC.append(epoch_te_SPC)
    te_PPV.append(epoch_te_PPV)
    te_NPV.append(epoch_te_NPV)
    te_F1.append(epoch_te_F1)
    "te_AUC.append(epoch_te_AUC)"  

0 : 0.7214583333333333
1 : 0.6064583333333333
2 : 0.7777083333333333
3 : 0.855
4 : 0.8783333333333333
5 : 0.8735416666666667
6 : 0.8829166666666667
7 : 0.8866666666666667
8 : 0.91
9 : 0.8660416666666667
10 : 0.8495833333333334
11 : 0.8925
12 : 0.9120833333333334
13 : 0.8902083333333334
14 : 0.8914583333333334
15 : 0.9054166666666666
16 : 0.8947916666666667
17 : 0.9052083333333333
18 : 0.910625
19 : 0.8922916666666667
20 : 0.92
21 : 0.9104166666666667
22 : 0.8791666666666667
23 : 0.9097916666666667
24 : 0.9185416666666667
25 : 0.9058333333333334
26 : 0.9077083333333333
27 : 0.9091666666666667
28 : 0.8954166666666666
29 : 0.91375
30 : 0.9129166666666667
31 : 0.9139583333333333
32 : 0.9029166666666667
33 : 0.909375
34 : 0.9072916666666667
35 : 0.905
36 : 0.895625
37 : 0.9166666666666666
38 : 0.9179166666666667
39 : 0.91
40 : 0.918125
41 : 0.906875
42 : 0.9035416666666667
43 : 0.9077083333333333
44 : 0.9164583333333334
45 : 0.9125
46 : 0.8779166666666667
47 : 0.90875
48 : 0.898333333333333

In [20]:
column_name = ['loss', 'accuracy', 'MCC', 'SE', 'SPC', 'PPV', 'NPV','F1']

tr_loss = pd.Series(tr_loss)
tr_accuracy = pd.Series(tr_accuracy)
tr_MCC = pd.Series(tr_MCC)
tr_SE = pd.Series(tr_SE)
tr_SPC = pd.Series(tr_SPC)
tr_PPV = pd.Series(tr_PPV)
tr_NPV = pd.Series(tr_NPV)
tr_F1 = pd.Series(tr_F1)

tr_result = pd.concat([tr_loss, tr_accuracy, tr_MCC, tr_SE, tr_SPC, tr_PPV, tr_NPV,tr_F1], axis=1)
tr_result.columns = column_name

In [21]:
column_name = ['loss', 'accuracy', 'MCC', 'SE', 'SPC', 'PPV', 'NPV','F1']

te_loss = pd.Series(te_loss)
te_accuracy = pd.Series(te_accuracy)
te_MCC = pd.Series(te_MCC)
te_SE = pd.Series(te_SE)
te_SPC = pd.Series(te_SPC)
te_PPV = pd.Series(te_PPV)
te_NPV = pd.Series(te_NPV)
te_F1= pd.Series(te_F1)

te_result = pd.concat([te_loss, te_accuracy, te_MCC, te_SE, te_SPC, te_PPV, te_NPV,te_F1], axis=1)
te_result.columns = column_name
te_result.index = [*range(1, epochs + 1)]

In [22]:
max(te_result.accuracy)

0.92

In [23]:
tr_loss = []
tr_accuracy = []
tr_MCC = []
tr_SE = []
tr_SPC = []
tr_PPV=[]
tr_NPV=[]
tr_AUC=[]
tr_F1=[]

te_loss = []
te_accuracy = []
te_MCC = []
te_SE = []
te_SPC = []
te_PPV=[]
te_NPV=[]
te_AUC=[]
te_F1=[]
from sklearn import metrics
epochs = 80

for epoch in range(epochs): 
    print(f'{epoch} : ',end = '')
    epoch_tr_loss, epoch_tr_accuracy, epoch_tr_MCC, epoch_tr_SE,epoch_tr_F1, epoch_tr_SPC,epoch_tr_PPV, epoch_tr_NPV,epoch_te_loss, epoch_te_accuracy, epoch_te_MCC, epoch_te_SE, epoch_te_SPC,epoch_te_PPV,epoch_te_F1,epoch_te_NPV= fit(model, optimizer, train_dl, test_dl)
    tr_loss.append(epoch_tr_loss)
    tr_accuracy.append(epoch_tr_accuracy)
    tr_MCC.append(epoch_tr_MCC)
    tr_SE.append(epoch_tr_SE)
    tr_SPC.append(epoch_tr_SPC)
    tr_PPV.append(epoch_tr_PPV)
    tr_NPV.append(epoch_tr_NPV)
    tr_F1.append(epoch_tr_F1)
    "tr_AUC.append(epoch_tr_AUC)"
    
    te_loss.append(epoch_te_loss)
    te_accuracy.append(epoch_te_accuracy)
    print(epoch_te_accuracy)
    te_MCC.append(epoch_te_MCC)
    te_SE.append(epoch_te_SE)
    te_SPC.append(epoch_te_SPC)
    te_PPV.append(epoch_te_PPV)
    te_NPV.append(epoch_te_NPV)
    te_F1.append(epoch_te_F1)
    "te_AUC.append(epoch_te_AUC)"  
column_name = ['loss', 'accuracy', 'MCC', 'SE', 'SPC', 'PPV', 'NPV','F1']

tr_loss = pd.Series(tr_loss)
tr_accuracy = pd.Series(tr_accuracy)
tr_MCC = pd.Series(tr_MCC)
tr_SE = pd.Series(tr_SE)
tr_SPC = pd.Series(tr_SPC)
tr_PPV = pd.Series(tr_PPV)
tr_NPV = pd.Series(tr_NPV)
tr_F1 = pd.Series(tr_F1)

tr_result = pd.concat([tr_loss, tr_accuracy, tr_MCC, tr_SE, tr_SPC, tr_PPV, tr_NPV,tr_F1], axis=1)
tr_result.columns = column_name

column_name = ['loss', 'accuracy', 'MCC', 'SE', 'SPC', 'PPV', 'NPV','F1']

te_loss = pd.Series(te_loss)
te_accuracy = pd.Series(te_accuracy)
te_MCC = pd.Series(te_MCC)
te_SE = pd.Series(te_SE)
te_SPC = pd.Series(te_SPC)
te_PPV = pd.Series(te_PPV)
te_NPV = pd.Series(te_NPV)
te_F1= pd.Series(te_F1)

te_result = pd.concat([te_loss, te_accuracy, te_MCC, te_SE, te_SPC, te_PPV, te_NPV,te_F1], axis=1)
te_result.columns = column_name
te_result.index = [*range(1, epochs + 1)]

max(te_result.accuracy)

0 : 0.9089583333333333
1 : 0.9104166666666667
2 : 0.87625
3 : 0.9054166666666666
4 : 0.9035416666666667
5 : 0.9041666666666667
6 : 0.90375
7 : 0.9091666666666667
8 : 0.909375
9 : 0.9047916666666667
10 : 0.9077083333333333
11 : 0.8933333333333333
12 : 0.8922916666666667
13 : 0.9045833333333333
14 : 0.8952083333333334
15 : 0.9047916666666667
16 : 0.9014583333333334
17 : 0.9060416666666666
18 : 0.898125
19 : 0.905
20 : 0.89875
21 : 0.8908333333333334
22 : 0.905
23 : 0.906875
24 : 0.8970833333333333
25 : 0.9022916666666667
26 : 0.905
27 : 0.8995833333333333
28 : 0.901875
29 : 0.9033333333333333
30 : 0.9020833333333333
31 : 0.9025
32 : 0.9070833333333334
33 : 0.9020833333333333
34 : 0.9025
35 : 0.9072916666666667
36 : 0.9020833333333333
37 : 0.90125
38 : 0.8979166666666667
39 : 0.9083333333333333
40 : 0.9029166666666667
41 : 0.899375
42 : 0.9039583333333333
43 : 0.9047916666666667
44 : 0.9029166666666667
45 : 0.9045833333333333
46 : 0.905
47 : 0.9010416666666666
48 : 0.9035416666666667
49 :

0.9104166666666667

In [23]:
# save
tr_result.to_csv(r'./调参前后/Word_embedding_GRUBI_train_result-初始.csv')
te_result.to_csv(r'./调参前后/Word_embedding_GRUBI_valid_result-初始.csv')

In [15]:
nn.GRU??